# comercio_ext_indices.tb_referencia_nbm_ncm
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NBM_NCM_delta`
## 📌 Descrição do arquivo

Tabela de referência (bridge) que mapeia **códigos NBM** (legado) para **códigos NCM** (atual/padrão do COMEX moderno).

Tipo: Bridge / Crosswalk de produto (NBM → NCM)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NBM`|Código do produto na NBM|Código legada do item|
|`CO_NCM`|Código do produto na NCM|Código padronizado atual equivalente|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/nbm_ncm"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_nbm/"
silverTable = "silver_comercio_ext_indices.tb_referencia_nbm_ncm"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_NBM",   T.StringType(),   False),
    T.StructField("CO_NCM",   T.StringType(),   False),
    T.StructField("TS_REF",   T.TimestampType(),False),
    T.StructField("NM_ORIGEM",T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_NBM", up("CO_NBM"))
    .withColumn("CO_NCM", up("CO_NCM"))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/comercio_ext_indices/NCM_NBM_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_NBM").isNotNull() & F.col("CO_NCM").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_NBM","CO_NCM"])

In [0]:
df_silver = df_dedup.select("CO_NBM","CO_NCM","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_ncm_nbm")

merge_condition = """
  t.CO_NBM   = s.CO_NBM
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.CO_NCM": "s.CO_NCM",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)